<a href="https://colab.research.google.com/github/alvaroamor/alvaroamor/blob/master/Elenatonta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import json
import ast
import yaml

In [2]:
def create_twitter_url():
    handle = "Elena25__"
    max_results = 100
    mrf = "max_results={}".format(max_results)
    q = "query=from:{}".format(handle)
    url = "https://api.twitter.com/2/tweets/search/all?{}&{}".format(
        mrf, q
    )
    return url

In [3]:
url = create_twitter_url()
print(url)

https://api.twitter.com/2/tweets/search/all?max_results=100&query=from:Elena25__


In [4]:
def process_yaml():
    with open("config.yaml") as file:
        return yaml.safe_load(file)

In [6]:
data = process_yaml()

In [7]:
def create_bearer_token(data):
    return data["search_tweets_api"]["bearer_token"]

In [8]:
bearer_token = create_bearer_token(data)

In [9]:
def twitter_auth_and_connect(bearer_token, url):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    response = requests.request("GET", url, headers=headers)
    return response.json()

In [10]:
res_json = twitter_auth_and_connect(bearer_token, url)
print(res_json)
if res_json == {'meta': {'result_count': 0}}:
    print("Check to see if the Twitter handle has Tweeted in the last 7 days")

{'data': [{'id': '1504496211717459973', 'text': 'RT @albaricoque_acg: Mi cabeza https://t.co/bkOI9e75EJ'}, {'id': '1503258942255902721', 'text': 'RT @albaricoque_acg: Necesidades https://t.co/LiC9biY8IE'}, {'id': '1499672168690589698', 'text': 'RT @jaaysart: The painting                  The palette https://t.co/1S8ZiYGiAr'}, {'id': '1499671447219933185', 'text': 'RT @maanso__: ir a la playa por la noche y escuchar las olas es todo lo que necesito ahora mismo'}, {'id': '1499345085338689538', 'text': 'Holi👋🏼\nHace un tiempo empecé a trabajar sobre un proyecto de abstracción basado en la belleza de la  figura humana. \nOs dejo el enlace a mi perfil por si queréis echarle un vistacillo y compartirlo🤩💖\nhttps://t.co/RjXioBavtj'}, {'id': '1499316606689808387', 'text': 'RT @AcademiaAesthe1: A magical show by a big murmuration of starlings.\n\n🎥 by Claire Droppert. https://t.co/dS26PENrFb'}, {'id': '1499006145079431175', 'text': 'RT @ImagenRetro: https://t.co/IT89CoEACL'}, {'id': '14990058918

In [11]:
def lang_data_shape(res_json):
    data_only = res_json["data"]
    doc_start = '"documents": {}'.format(data_only)
    str_json = "{" + doc_start + "}"
    dump_doc = json.dumps(str_json)
    doc = json.loads(dump_doc)
    return ast.literal_eval(doc)

In [12]:
documents = lang_data_shape(res_json)
print(documents)

{'documents': [{'id': '1504496211717459973', 'text': 'RT @albaricoque_acg: Mi cabeza https://t.co/bkOI9e75EJ'}, {'id': '1503258942255902721', 'text': 'RT @albaricoque_acg: Necesidades https://t.co/LiC9biY8IE'}, {'id': '1499672168690589698', 'text': 'RT @jaaysart: The painting                  The palette https://t.co/1S8ZiYGiAr'}, {'id': '1499671447219933185', 'text': 'RT @maanso__: ir a la playa por la noche y escuchar las olas es todo lo que necesito ahora mismo'}, {'id': '1499345085338689538', 'text': 'Holi👋🏼\nHace un tiempo empecé a trabajar sobre un proyecto de abstracción basado en la belleza de la  figura humana. \nOs dejo el enlace a mi perfil por si queréis echarle un vistacillo y compartirlo🤩💖\nhttps://t.co/RjXioBavtj'}, {'id': '1499316606689808387', 'text': 'RT @AcademiaAesthe1: A magical show by a big murmuration of starlings.\n\n🎥 by Claire Droppert. https://t.co/dS26PENrFb'}, {'id': '1499006145079431175', 'text': 'RT @ImagenRetro: https://t.co/IT89CoEACL'}, {'id': '149900

In [13]:
def connect_to_azure(data):
    azure_url = "https://tweetmining2.cognitiveservices.azure.com/"
    language_api_url = "{}text/analytics/v2.1/languages".format(azure_url)
    sentiment_url = "{}text/analytics/v2.1/sentiment".format(azure_url)
    subscription_key = data["azure"]["subscription_key"]
    return language_api_url, sentiment_url, subscription_key

In [14]:
language_api_url, sentiment_url, subscription_key = connect_to_azure(data)

In [15]:
def azure_header(subscription_key):
    return {"Ocp-Apim-Subscription-Key": subscription_key}

In [16]:
headers = azure_header(subscription_key)

In [17]:
def generate_languages(headers, language_api_url, documents):
    response = requests.post(language_api_url, headers=headers, json=documents)
    return response.json()

In [18]:
with_languages = generate_languages(headers, language_api_url, documents)
print(with_languages)

{'documents': [{'id': '1504496211717459973', 'detectedLanguages': [{'name': 'Spanish', 'iso6391Name': 'es', 'score': 0.8333333730697632}]}, {'id': '1503258942255902721', 'detectedLanguages': [{'name': 'Spanish', 'iso6391Name': 'es', 'score': 0.800000011920929}]}, {'id': '1499672168690589698', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1499671447219933185', 'detectedLanguages': [{'name': 'Spanish', 'iso6391Name': 'es', 'score': 0.949999988079071}]}, {'id': '1499345085338689538', 'detectedLanguages': [{'name': 'Spanish', 'iso6391Name': 'es', 'score': 1.0}]}, {'id': '1499316606689808387', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1499006145079431175', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1499005891856809985', 'detectedLanguages': [{'name': 'Spanish', 'iso6391Name': 'es', 'score': 1.0}]}, {'id': '1499005152648482824', 'detectedLanguages': [{'name': '

In [21]:
def combine_lang_data(documents, with_languages):
    langs = pd.DataFrame(with_languages["documents"])
    lang_iso = [x.get("iso6391Name")
                for d in langs.detectedLanguages if d for x in d]
    data_only = documents["documents"]
    tweet_data = pd.DataFrame(data_only)
    tweet_data.insert(2, "language", lang_iso, True)
    json_lines = tweet_data.to_json(orient="records")
    return json_lines

In [22]:
json_lines = combine_lang_data(documents, with_languages)
print(json_lines)

[{"id":"1504496211717459973","text":"RT @albaricoque_acg: Mi cabeza https:\/\/t.co\/bkOI9e75EJ","language":"es"},{"id":"1503258942255902721","text":"RT @albaricoque_acg: Necesidades https:\/\/t.co\/LiC9biY8IE","language":"es"},{"id":"1499672168690589698","text":"RT @jaaysart: The painting                  The palette https:\/\/t.co\/1S8ZiYGiAr","language":"en"},{"id":"1499671447219933185","text":"RT @maanso__: ir a la playa por la noche y escuchar las olas es todo lo que necesito ahora mismo","language":"es"},{"id":"1499345085338689538","text":"Holi\ud83d\udc4b\ud83c\udffc\nHace un tiempo empec\u00e9 a trabajar sobre un proyecto de abstracci\u00f3n basado en la belleza de la  figura humana. \nOs dejo el enlace a mi perfil por si quer\u00e9is echarle un vistacillo y compartirlo\ud83e\udd29\ud83d\udc96\nhttps:\/\/t.co\/RjXioBavtj","language":"es"},{"id":"1499316606689808387","text":"RT @AcademiaAesthe1: A magical show by a big murmuration of starlings.\n\n\ud83c\udfa5 by Claire Droppert.

In [23]:
def add_document_format(json_lines):
    docu_format = '"' + "documents" + '"'
    json_docu_format = "{}:{}".format(docu_format, json_lines)
    docu_align = "{" + json_docu_format + "}"
    jd_align = json.dumps(docu_align)
    jl_align = json.loads(jd_align)
    return ast.literal_eval(jl_align)

In [24]:
document_format = add_document_format(json_lines)
print(document_format)

{'documents': [{'id': '1504496211717459973', 'text': 'RT @albaricoque_acg: Mi cabeza https:\\/\\/t.co\\/bkOI9e75EJ', 'language': 'es'}, {'id': '1503258942255902721', 'text': 'RT @albaricoque_acg: Necesidades https:\\/\\/t.co\\/LiC9biY8IE', 'language': 'es'}, {'id': '1499672168690589698', 'text': 'RT @jaaysart: The painting                  The palette https:\\/\\/t.co\\/1S8ZiYGiAr', 'language': 'en'}, {'id': '1499671447219933185', 'text': 'RT @maanso__: ir a la playa por la noche y escuchar las olas es todo lo que necesito ahora mismo', 'language': 'es'}, {'id': '1499345085338689538', 'text': 'Holi\ud83d\udc4b\ud83c\udffc\nHace un tiempo empecé a trabajar sobre un proyecto de abstracción basado en la belleza de la  figura humana. \nOs dejo el enlace a mi perfil por si queréis echarle un vistacillo y compartirlo\ud83e\udd29\ud83d\udc96\nhttps:\\/\\/t.co\\/RjXioBavtj', 'language': 'es'}, {'id': '1499316606689808387', 'text': 'RT @AcademiaAesthe1: A magical show by a big murmuration of st

In [27]:
def sentiment_scores(headers, sentiment_url, document_format):
    response = requests.post(
        sentiment_url, headers=headers, json=document_format)
    return response.json()

In [28]:
sentiments = sentiment_scores(headers, sentiment_url, document_format)
print(sentiments)

{'documents': [{'id': '1499672168690589698', 'score': 0.5}, {'id': '1499316606689808387', 'score': 0.9717398881912231}, {'id': '1499006145079431175', 'score': 0.5}, {'id': '1499005152648482824', 'score': 0.5}, {'id': '1496028145446756353', 'score': 0.5}, {'id': '1504496211717459973', 'score': 0.3211559057235718}, {'id': '1503258942255902721', 'score': 0.5}, {'id': '1499671447219933185', 'score': 0.493371844291687}, {'id': '1499345085338689538', 'score': 0.7702702879905701}, {'id': '1499005891856809985', 'score': 0.5}, {'id': '1498344733596430338', 'score': 0.4910581707954407}, {'id': '1498195206151225354', 'score': 0.7482664585113525}, {'id': '1498194926026244099', 'score': 0.596170961856842}, {'id': '1497908536663236608', 'score': 0.5206627249717712}, {'id': '1497856043598102531', 'score': 0.5769621133804321}, {'id': '1497855390620565508', 'score': 0.5}, {'id': '1497261974782201863', 'score': 0.5080981850624084}, {'id': '1495559495522701316', 'score': 0.3824429512023926}, {'id': '1493

In [29]:
def mean_score(sentiments):
    sentiment_df = pd.DataFrame(sentiments["documents"])
    return sentiment_df["score"].mean()

In [30]:
week_score = mean_score(sentiments)
print(week_score)

0.547536301612854


In [241]:
def no_tweets(res_json):
    if res_json == {"meta": {"result_count": 0}}:
        print("The Twitter handle entered hasn't Tweeted in 7 days.")

In [242]:
def connect_to_azure(data):
    azure_url = "https://week.cognitiveservices.azure.com/"
    language_api_url = "{}text/analytics/v2.1/languages".format(azure_url)
    sentiment_url = "{}text/analytics/v2.1/sentiment".format(azure_url)
    subscription_key = data["azure"]["subscription_key"]
    return sentiment_url, subscription_key

In [243]:
def azure_header(subscription_key):
    return {"Ocp-Apim-Subscription-Key": subscription_key}

In [244]:
def create_document_format(res_json):
    data_only = res_json["data"]
    doc_start = '"documents": {}'.format(data_only)
    str_json = "{" + doc_start + "}"
    dump_doc = json.dumps(str_json)
    doc = json.loads(dump_doc)
    return ast.literal_eval(doc)

In [245]:
def sentiment_scores(headers, sentiment_url, document_format):
    response = requests.post(
        sentiment_url, headers=headers, json=document_format)
    return response.json()

In [246]:
def mean_score(sentiments):
    sentiment_df = pd.DataFrame(sentiments['documents'])
    return sentiment_df["score"].mean()

In [247]:
def week_logic(week_score):
    if week_score > 0.75 or week_score == 0.75:
        print("You had a positive week")
    elif week_score > 0.45 or week_score == 0.45:
        print("You had a neutral week")
    else:
        print("You had a negative week, I hope it gets better")

In [248]:
def main():
    print("Hola1")
    url = create_twitter_url()
    data = process_yaml()
    bearer_token = create_bearer_token(data)
    res_json = twitter_auth_and_connect(bearer_token, url)
    no_tweets(res_json)
    sentiment_url, subscription_key = connect_to_azure(data)
    headers = azure_header(subscription_key)
    document_format = create_document_format(res_json)
    sentiments = sentiment_scores(headers, sentiment_url, document_format)
    week_score = mean_score(sentiments)
    print(week_score)
    week_logic(week_score)
    print("Hola2")

In [249]:
if __name__ == "__main__":
    main()

Hola1


KeyError: ignored

In [ ]:
pip install panda

# Nueva sección